This notebook can be used to generate/evaluate coreference resolution

In [ ]:
import pandas as pd

# for example load evidences
df_sentences = pd.read_csv("datasets_hover_llama/mini_fact_hover.pkl")

In [ ]:
import spacy
from fastcoref import spacy_component

nlp = spacy.load("en_core_web_sm")

nlp.add_pipe(
    "fastcoref", 
    config={
        'model_architecture': 'LingMessCoref', 
        'model_path': 'biu-nlp/lingmess-coref', 
        'device': 'cuda' 
    }
)

In [3]:
def convert_text_to_sentences(text):
    all_sentences = []
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    all_sentences.append(sentences)
    all_sentences = [item for sublist in all_sentences for item in sublist]
    return all_sentences

In [ ]:
from fastcoref import spacy_component
import spacy

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe(
   "fastcoref", 
   config={'model_architecture': 'LingMessCoref', 'model_path': 'biu-nlp/lingmess-coref', 'device': 'cuda'}
)

In [ ]:
import random

coref_samples = []
raw_samples = []
gpt_samples = []

df_evidence = df_sentences.groupby('gen_evidence')
evidence_groups = list(df_evidence)
rng = random.Random(42)


rng.shuffle(evidence_groups)

for gen_evidence, group in evidence_groups:
    gen_evidence_with_coref = group['gen_evidence_with_coref'].values[0]
    gen_evidence_sentences = convert_text_to_sentences(gen_evidence)
    
    if str(gen_evidence_with_coref).strip() != str(gen_evidence).strip():
        if len(gen_evidence_sentences) > 1:
            print(gen_evidence)
            print(gen_evidence_with_coref)
            
            doc = nlp(      
                gen_evidence, 
                component_cfg={"fastcoref": {'resolve_text': True}}
            )

            resolved_text = doc._.resolved_text
            print(resolved_text)
            
            raw_samples.append(gen_evidence)
            coref_samples.append(resolved_text)
            gpt_samples.append(gen_evidence_with_coref)

            if len(coref_samples) == 100:
                break


In [34]:
df = pd.DataFrame({
    'Raw Sample': raw_samples,
    'Coref Edit': coref_samples,
    'GPT Edit': gpt_samples,
})

df.to_excel('gpt_coref.xlsx', index=False)

In [7]:
import pandas as pd 

df = pd.read_excel('gpt_coref.xlsx')

df = df.loc[(df['Correct Coref'] != 2) & (df['Correct GPT'] != 2)].iloc[:1]

In [8]:
import pandas as pd 

df = pd.read_excel('gpt_coref.xlsx')

#df2 = df.loc[(df['Correct GPT'] == 2)]
#print(len(df2))

#df = df.loc[(df['Correct Coref'] != 2) & (df['Correct GPT'] != 2)]

print(len(df))

df_gpt = df.loc[(df['Correct GPT'] == 1) | (df['Correct GPT'] == 2)]
df_coref = df.loc[(df['Correct Coref'] == 1) ]

print(len(df_gpt))
print(len(df_gpt) / len(df))

print(len(df_coref))
print(len(df_coref) / len(df))


500
488
0.976
103
0.206
